In [401]:
libraries = c("dplyr", "tidyverse", "data.table", "readxl")
for(x in libraries) {library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE)}

theme_set(theme_bw())

In [402]:
path <- "../data/flight/all_region/"

options(warn=-1)
list.files(path = path, pattern = "*xlsx") -> file_list
read_xlsx_files <- function(x){
    df <- read_xlsx(path = paste0(path, x), skip = 5)
    df[-1, c(1,2,27)] -> df_2019
    colnames(df_2019) <- c("destination", "series", "v2019")
    return(df_2019)}
lapply(file_list, read_xlsx_files ) -> df_list

for(i in 1:length(df_list)){
    colnames(df_list[[i]]) <- c("destination", "series", substr(file_list[i],1,nchar(file_list[i])-5))
}

options(warn=0)

New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
•

New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`


In [403]:
merge.all <- function(x, y) {merge(x, y, all=TRUE, by=c("destination", "series"))}
output <- Reduce(merge.all, df_list)

In [404]:
output %>% filter(!(destination %in% c("END", "Türkiye", "Czechia"))) %>%
mutate(destination=case_when(destination==c("Bolivia, Plurinational State of")~c("Bolivia"), 
                             TRUE~destination)) -> output

In [405]:
new_list <- list()
unique(output$destination) -> country_list

for (g in 1:length(country_list)){
    output %>% filter(destination==country_list[g]) -> temp
    if(nrow(temp)>=2){
        if(any(temp$series==c("VFR"))){temp %>% filter(series==c("VFR")) -> new_list[[g]]}
        else if(any(temp$series==c("VFN")) && !(temp$series %in% c("VFR")))
        {temp %>% filter(series==c("VFN")) -> new_list[[g]]}
        else if(any(temp$series==c("TFR")) && !(temp$series %in% c("VFR", "VFN")))
        {temp %>% filter(series==c("TFR")) -> new_list[[g]]}
        else if(any(temp$series==c("TFN")) && !(temp$series %in% c("VFR", "VFR", "TFR")))
        {temp %>% filter(series==c("TFN")) -> new_list[[g]]}
        else if(any(temp$series==c("TCER")) && !(temp$series %in% c("VFR", "VFR", "TFR", "TFN")))
        {temp %>% filter(series==c("TCER")) -> new_list[[g]]}
        else if(any(temp$series==c("TCEN")) && !(temp$series %in% c("VFR", "VFR", "TFR", "TFN", "TCER")))
        {temp %>% filter(series==c("TCEN")) -> new_list[[g]]}
        else if(any(temp$series==c("THSR")) && !(temp$series %in% c("VFR", "VFR", "TFR", "TFN", "TCER", "TCEN")))
        {temp %>% filter(series==c("THSR")) -> new_list[[g]]}
        else if(any(temp$series==c("THSN")) && !(temp$series %in% c("VFR", "VFR", "TFR", "TFN", "TCER", "TCEN", "THSR")))
        {temp %>% filter(series==c("THSN")) -> new_list[[g]]}
    }
    else{temp -> new_list[[g]]}
}
            
do.call("rbind", new_list) -> flight_matrix
write.csv(flight_matrix, "../data/flight/flight_matrix.csv")     

In [406]:
write.csv(flight_matrix, "../data/flight/flight_matrix.csv") 

In [407]:
setdiff(flight_matrix$destination, colnames(flight_matrix[,3:214]))

[1] "Antigua and Barbuda"              "Bosnia and Herzegovina"          
 [3] "Cabo Verde"                       "Curaçao"                         
 [5] "Côte d'Ivoire"                    "Eswatini"                        
 [7] "French Guiana"                    "Lao People's Democratic Republic"
 [9] "Micronesia, Federated States of"  "North Macedonia"                 
[11] "Reunion"                          "Saint Kitts and Nevis"           
[13] "Saint Vincent and the Grenadines" "Sint Maarten (Dutch part)"       
[15] "Taiwan Province of China"         "Tanzania, United Republic of"    
[17] "Timor-Leste"                      "Trinidad and Tobago"             
[19] "Turks and Caicos Islands"         "United States Virgin Islands"

In [395]:
read.csv("../data/MSM_pop/df_all_inci.csv") -> df_all_inci
df_all_inci %>% 
mutate(location=case_when(location==c("Democratic Republic of Congo")~c("Congo, Democratic Republic of the"),
                          location==c("Curacao")~c("Curaçao"),
                          location==c("Czechia")~c("Czech Republic"),
                          location==c("Iran")~c("Iran, Islamic Republic of"),
                          location==c("South Korea")~c("Korea, Republic of"),
                          location==c("United States")~c("United States of America"),
                          location==c("Venezuela")~c("Venezuela, Bolivarian Republic of"),
                          location==c("Moldova")~c("Moldova, Republic of"),
                          location==c("Russia")~c("Russian Federation"),
                          TRUE~location)) -> df_all_inci

is.element(unique(df_all_inci$location), flight_matrix$destination) -> temp
which(temp==c("FALSE"))
unique(df_all_inci$location)[42]
unique(df_all_inci$location)[43]
unique(df_all_inci$location)[44]
unique(df_all_inci$location)[46]
unique(df_all_inci$location)[65]
unique(df_all_inci$location)[69]
unique(df_all_inci$location)[93]

[1] 42 43 44 46 65 69 93

[1] "Ghana"

[1] "Gibraltar"

[1] "Guadeloupe"

[1] "Greenland"

[1] "Liberia"

[1] "Saint Martin (French part)"

[1] "Sudan"